In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import torchvision.datasets as dset
import torchvision.transforms as T

# GenerateDataset

In [3]:
def imbalanceBalancedDataset(ds, weights):
    targets = np.array(ds.targets)
    classes, class_counts = np.unique(targets, return_counts=True)
    nb_classes = len(classes)
    weights = list(np.int_(weights * (class_counts[0])))
    class_indices = [np.where(targets == i)[0] for i in range(nb_classes)]
    # Get imbalanced number of instances
    imbal_class_indices = [class_idx[:class_count] for class_idx, class_count in zip(class_indices, weights)]
    imbal_class_indices = np.hstack(imbal_class_indices)
    
    # Set target and data to dataset
    ds.targets = targets[imbal_class_indices]
    ds.data = ds.data[imbal_class_indices]
    
    return ds

NUM_TRAIN = 44000
NUM_VAL = 6000
WEIGHTS = np.array([0.0, 0.0, 0.0, 0.1, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0])

cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=T.ToTensor())
cifar10_train.data = cifar10_train.data[:NUM_TRAIN]
cifar10_train.targets = cifar10_train.targets[:NUM_TRAIN]
imbalanceBalancedDataset(cifar10_train, WEIGHTS);

Files already downloaded and verified


In [4]:
dataset_ = cifar10_train

In [12]:
from smrt.smrt.balance.smrt import smrt_balance

/opt/conda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
W0719 07:19:26.803178 139801215850240 deprecation_wrapper.py:119] From /data/octavf/workspace/cifar/smrt/smrt/autoencode/autoencoder.py:39: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W0719 07:19:26.804279 139801215850240 deprecation_wrapper.py:119] From /data/octavf/workspace/cifar/smrt/smrt/autoencode/autoencoder.py:40: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W0719 07:19:26.805139 139801215850240 deprecation_wrapper.py:119] From /data/octavf/workspace/cifar/smrt/smrt/autoencode/autoencoder.py:41: The name tf.train.Ada

In [5]:
x_shape = dataset_.data.shape
dataset_.data = dataset_.data.reshape(-1, x_shape[1] * x_shape[2] * x_shape[3])

In [6]:
targets = np.array(dataset_.targets)
classes, class_counts = np.unique(targets, return_counts=True)
classes, class_counts

(array([3, 5, 6]), array([ 439, 4359, 4391]))

In [7]:
dataset_.data.shape, dataset_.targets.shape

((9189, 3072), (9189,))

In [8]:
test_x = dataset_.data[dataset_.targets == 3, :]
test_x.shape

(439, 3072)

In [9]:
test_x[0][:]

array([125, 125, 116, ..., 144, 116,  86], dtype=uint8)

In [29]:
smrted = smrt_balance(dataset_.data, dataset_.targets,
                      n_hidden=900,#[500,100,100,200],
                      n_latent_factors=10,
                      n_epochs=36,
                      random_state=42,
                      balance_ratio=1.0,
                      shuffle=False,
                      activation_function='relu',
                      early_stopping=True,
                      dropout=0.3,
                      batch_size=64,
                      learning_function='adam');

/home/docker/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


needed obs 3952 via VariationalAutoEncoder(activation_function='relu', batch_size=64,
                       bias_strategy='zeros', clip=True, display_step=5,
                       dropout=0.3, early_stopping=True, eps=1e-10,
                       gclip_max=5.0, gclip_min=-5.0, l2_penalty=0.0001,
                       layer_type='xavier', learning_function='adam',
                       learning_rate=0.05, min_change=0.001, n_epochs=36,
                       n_hidden=900, n_latent_factors=10,
                       random_state=<smrt.smrt.utils.SeededRandomState object at 0x7f255c2912e8>,
                       verbose=0)


/home/docker/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


needed obs 32 via VariationalAutoEncoder(activation_function='relu', batch_size=64,
                       bias_strategy='zeros', clip=True, display_step=5,
                       dropout=0.3, early_stopping=True, eps=1e-10,
                       gclip_max=5.0, gclip_min=-5.0, l2_penalty=0.0001,
                       layer_type='xavier', learning_function='adam',
                       learning_rate=0.05, min_change=0.001, n_epochs=36,
                       n_hidden=900, n_latent_factors=10,
                       random_state=<smrt.smrt.utils.SeededRandomState object at 0x7f255c2912e8>,
                       verbose=0)


In [31]:
smrted[0][-1]

array([nan, nan, nan, nan, ..., nan, nan, nan, nan])

# Debug

In [10]:
from smrt.smrt.autoencode import VariationalAutoEncoder

v_encoder = VariationalAutoEncoder(n_epochs=100, n_hidden=[100,100,100], n_latent_factors=3, 
                                   learning_rate=0.5, batch_size=64, display_step=25, 
                                   activation_function='sigmoid', verbose=2, l2_penalty=None,
                                   random_state=42, dropout=0.2, #early_stopping=True,
                                   learning_function='adam', clip=False)

v_encoder.fit(test_x)

/opt/conda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
W0719 07:38:34.692059 140040194025216 deprecation_wrapper.py:119] From /data/octavf/workspace/cifar/smrt/smrt/autoencode/autoencoder.py:39: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W0719 07:38:34.693703 140040194025216 deprecation_wrapper.py:119] From /data/octavf/workspace/cifar/smrt/smrt/autoencode/autoencoder.py:40: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W0719 07:38:34.694660 140040194025216 deprecation_wrapper.py:119] From /data/octavf/workspace/cifar/smrt/smrt/autoencode/autoencoder.py:41: The name tf.train.Ada

Epoch: 1, cost=nan, time=0.2196 (sec)
Epoch: 26, cost=nan, time=0.1036 (sec)
Epoch: 51, cost=nan, time=0.1016 (sec)
Epoch: 76, cost=nan, time=0.1014 (sec)
Optimization complete after 100 epoch(s). Average epoch time: 0.1024 seconds


VariationalAutoEncoder(activation_function='sigmoid', batch_size=64,
                       bias_strategy='zeros', clip=False, display_step=25,
                       dropout=0.2, early_stopping=False, eps=1e-09,
                       gclip_max=5.0, gclip_min=-5.0, l2_penalty=None,
                       layer_type='xavier', learning_function='adam',
                       learning_rate=0.5, min_change=0.001, n_epochs=100,
                       n_hidden=[3072, 100, 100, 100], n_latent_factors=3,
                       random_state=42, verbose=2)

In [14]:
fake = v_encoder.generate_from_sample(test_x[:10])
fake

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [15]:
fake = v_encoder.generate()
fake

array([[nan, nan, nan, nan, ..., nan, nan, nan, nan]], dtype=float32)